In [1]:
import cv2
import numpy as np
import math
from math import atan2, degrees

In [2]:
def resize_image(image, multiple):
    height, width = image.shape[:2]
    new_height = height + (multiple - height % multiple) % multiple
    new_width = width + (multiple - width % multiple) % multiple
    resized_image = cv2.resize(image, (new_width, new_height))
    return resized_image

In [3]:
def compute_gradients(image):
    dx = cv2.filter2D(image, -1, np.array([[-1, 0, 1]]))
    dy = cv2.filter2D(image, -1, np.array([[-1], [0], [1]]))
    magnitude = np.sqrt(dx**2 + dy**2)
    orientation = np.degrees(np.arctan2(dx, dy))
    return magnitude, orientation

In [4]:
def compute_histograms(orientation, magnitude, cells_per_block=2, bins=9):
    cell_size = 8
    height, width = orientation.shape[:2]
    cell_histograms = []
    for i in range(0, height, cell_size):
        for j in range(0, width, cell_size):
            cell_orientation = orientation[i:i+cell_size, j:j+cell_size]
            cell_magnitude = magnitude[i:i+cell_size, j:j+cell_size]
            histogram = np.zeros(bins)
            for k in range(cell_size):
                for l in range(cell_size):
                    bin_index = int(cell_orientation[k, l] // (360/bins))
                    histogram[bin_index] += cell_magnitude[k, l]
            cell_histograms.append(histogram)
    block_histograms = []
    for i in range(0, len(cell_histograms), cells_per_block*cells_per_block):
        block_histogram = np.concatenate(cell_histograms[i:i+cells_per_block*cells_per_block])
        block_histograms.append(block_histogram)
    return block_histograms

In [5]:
def normalize_histograms(histograms):
    normalized_histograms = []
    for histogram in histograms:
        squared_sum = np.sqrt(np.sum(histogram**2))
        if squared_sum != 0:
            normalized_histogram = histogram / squared_sum
        else:
            normalized_histogram = histogram
        normalized_histograms.append(normalized_histogram)
    return normalized_histograms


In [6]:
def compute_hog(image):
    resized_image = resize_image(image, 8)
    magnitude, orientation = compute_gradients(resized_image)
    histograms = compute_histograms(orientation, magnitude)
    normalized_histograms = normalize_histograms(histograms)
    hog = np.concatenate(normalized_histograms)
    return hog

In [7]:
image = cv2.imread('0.jpg', 0)
hog = compute_hog(image)

print("HOG:")
print(hog)

HOG:
[0.29137584 0.13927667 0.27968381 ... 0.         0.         0.        ]


In [8]:

image = cv2.imread('0.jpg', cv2.IMREAD_GRAYSCALE)

hog = compute_hog(image)


np.savetxt('hog_descriptor.txt', hog)

# Visualizar orientações nos histogramas das células
resized_image = resize_image(image, 8)
magnitude, orientation = compute_gradients(resized_image)
histograms = compute_histograms(orientation, magnitude)

# Desenhar orientações nos histogramas das células e salvar as imagens
cell_size = 8
bins = 9
for i in range(min(len(histograms), 5)):
    histogram = histograms[i]
    center_x = (i % (resized_image.shape[1] // cell_size)) * cell_size + cell_size // 2
    center_y = (i // (resized_image.shape[1] // cell_size)) * cell_size + cell_size // 2
    visualization_image = cv2.cvtColor(resized_image, cv2.COLOR_GRAY2BGR)
    for j in range(bins):
        angle = j * (180 / bins)
        length = int(histogram[j] * cell_size * 3)
        x = int(center_x + length * math.cos(math.radians(angle)))
        y = int(center_y + length * math.sin(math.radians(angle)))
        cv2.line(visualization_image, (center_x, center_y), (x, y), (0, 0, 255), 1)
    cv2.imwrite(f'orientation_visualization_{i+1}.jpg', visualization_image)

print("Descritor HoG calculado e imagens salvas com sucesso!")

Descritor HoG calculado e imagens salvas com sucesso!
